# Создание решения по стандартизации названий спортивных школ.

**Описание проекта**

<br>Сервис "Мой Чемпион" помогает спортивным школам фигурного катания, тренерам
<br>мониторить результаты своих подопечных и планировать дальнейшее развитие спортсменов.

**Цель**

- Создать решение для стандартизации названий спортивных школ.
  <br>Например, одна и та же школа может быть записана по-разному
  <br>Необхдодимо сопоставить эти варианты эталонному названию из предоставленной таблицы

**Задачи**

- Изучить данные – эталонные названия СШ и варианты пользовательского ввода
- Подготовить обучающий набор данных на основе эталонного датасета
- Создать модель для подбора наиболее вероятных названий при ошибочном вводе
- Создать функцию (класс, модуль) для применения в сервисе
  - возможность выбора количества кандидатов
  - вывод в виде списка словарей
- Протестировать решение
- Проанализировать результат и предложить варианты улучшения
- Создать документацию
  - описание признаков
  - какая модель используется
  - как оценивается качество
  - инструкция по запуску (применению)
- Создать демо приложение

## Установка зависимостей

In [1]:
# !pip install pandas
# !pip install seaborn
# !pip install phik
# !pip install category_encoders
# !pip install python-dotenv
# !pip install mlflow
# !pip install psycopg2-binary
# !pip install autofeat
# !pip install optuna
# !pip install lightgbm
# !pip freeze > requirements.txt

In [2]:
# Импорт стандартных библиотек
import os

# Импорт библиотек для визуализации данных
# import matplotlib.pyplot as plt
# import seaborn as sns
# from matplotlib.gridspec import GridSpec

# # Импорт библиотек для работы с данными
# import pandas as pd
# import numpy as np

# # Импорт дополнительных библиотек
# import joblib
# import mlflow
# from phik.report import plot_correlation_matrix

# # Импорт пользовательских функций
# from research_functions import (
#     data_review,
#     data_preprocessing,
#     model_fitting,
#     model_logging,
#     models_comparison,
#     test_best_model
#     )

In [3]:
# Автоматическая перезагрузка модулей
%load_ext autoreload
%autoreload 2

# Импорт модуля research_functions
# import research_functions

Объявление глобальных переменных

In [4]:
DATA_DIR = 'data'
ASSETS_DIR = 'assets'
RANDOM_STATE = 42
# TEST_SIZE = 0.1

# pd.options.display.max_columns = 100
# pd.options.display.max_rows = 64
# pd.options.mode.copy_on_write = True

# sns.set_theme(style='white', palette='husl')

# %matplotlib inline
# %config InlineBackend.figure_format = 'png'
# %config InlineBackend.figure_format = 'retina'

## Знакомство с данными